In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from t5.data import preprocessors as prep
import functools
import t5
import gin
import sentencepiece as spm
from glob import glob
import os

gin.parse_config_file('pretrained_models_base_operative_config.gin')
vocab = 'sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

True

In [3]:
import json

with open('../../news/populate-news.json') as fopen:
    data = json.load(fopen)

In [5]:
data.keys()

dict_keys(['text', 'title'])

In [8]:
from tqdm import tqdm
import re

def cleaning(string):
    string = string.replace('\n', ' ').replace('\t', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [7]:
len(data['title'][0].split())

12

In [9]:
with tf.io.gfile.GFile('newstitle.tsv', "w") as outfile:
    for i in range(len(data['title'])):
        outfile.write("%s\t%s\n" % (cleaning(data['text'][i]), 
                                    cleaning(data['title'][i])))

In [10]:
def news_dataset(split, shuffle_files = False):
    del shuffle_files
    ds = tf.data.TextLineDataset(
        [
            'newstitle.tsv'
        ]
    )

    ds = ds.map(
        functools.partial(
            tf.io.decode_csv,
            record_defaults = ['', ''],
            field_delim = '\t',
            use_quote_delim = False,
        ),
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )
    ds = ds.map(lambda *ex: dict(zip(['question', 'answer'], ex)))
    return ds

def news_preprocessor(ds):
    def to_inputs_and_targets(ex):
        return {
            'inputs': tf.strings.join(['tajuk: ', ex['question']]),
            'targets': ex['answer'],
        }

    return ds.map(
        to_inputs_and_targets,
        num_parallel_calls = tf.data.experimental.AUTOTUNE,
    )

In [11]:
t5.data.TaskRegistry.remove('news_dataset')
t5.data.TaskRegistry.add(
    'news_dataset',
    dataset_fn = news_dataset,
    splits = ['train'],
    text_preprocessor = [news_preprocessor],
    sentencepiece_model_path = vocab,
    metric_fns = [t5.evaluation.metrics.accuracy],
)


/home/husein/.local/lib/python3.6/site-packages/t5/models/mesh_transformer.py:210: UserWarning: get_sentencepiece_model_path is deprecated. Please pass the mixture or task vocabulary directly to the Mesh TensorFlow Transformer instead.
  "get_sentencepiece_model_path is deprecated. Please pass the mixture or "


In [12]:
nq_task = t5.data.TaskRegistry.get("news_dataset")
ds = nq_task.get_dataset(split='qa.tsv', sequence_length={"inputs": 1024, "targets": 1024})
r = tfds.as_numpy(ds)

In [13]:
next(r)

{'inputs_plaintext': b'tajuk: Aplikasi Zoom mengalami lonjakan pengguna karena banyak orang menggunakannya selama masa pandemi COVID-19. Namun di samping itu, isu keamanan membuatnya dilarang di sejumlah negara dan lembaga. Sejumlah isu keamanan yang menghantui pengguna Zoom antara lain penyusupan orang tidak dikenal saat video conference (Zoom Bombing), data pengguna bocor, hingga panggilan yang tidak dienkripsi end to end. Kelemahan keamanan ini memicu sejumlah organisasi, perusahaan, pemerintah, lembaga pemerintah, dan sekolah melarang Zoom atau membatasi penggunaannya. Berikut ini daftarnya. Perusahaan Google melarang karyawannya menggunakan Zoom. Raksasa internet ini bahkan memerintahkan administrator langsung menonaktifkan Zoom yang dipakai di perangkat milik karyawan. Sebagai gantinya, karyawan Google diarahkan untuk menggunakan Google Duo SpaceX, dengan alasan keamanan dan privasi, melarang karyawan menggunakan Zoom Smart Communications, Internet Service Provider yang berbasis 